In [1]:
import investpy as ip
from datetime import datetime
import numpy as np
from tqdm import tqdm
import pandas as pd
import quantstats as qs
import warnings
warnings.filterwarnings('ignore')

In [2]:
today = datetime.today().date().strftime('%d/%m/%Y')
start_of_year = f'1/1/{datetime.today().date().year}'

In [3]:
def get_prices(assets,
                      amounts,
                      types = None,
                      rf=0.02,
                      num_sim = 5000,
                      from_date = start_of_year,
                      to_date = today,
                      last_month=False, #overrides from_date and to_date
                      show_current_performance=True, 
                      plot_frontier=True,
                      plot_simulations=True,
                      plot_stocks=True):
    
    if types == None:
        types = ['stock' for i in range(len(assets))]
    
    assert len(assets) == len(amounts) == len(types)
    
    #Create allocation dataframe
    alloc = pd.DataFrame({'Name': assets, 'Allocation': amounts})
    alloc['Allocation'] = alloc.Allocation / sum(alloc.Allocation)
    assert sum(alloc.Allocation)
    
    #Get prices
    prices = pd.DataFrame([], columns = assets)
    for i, s in tqdm(enumerate(assets)):
        if last_month:
            if types[i] == 'stock':
                prices[s] = ip.get_stock_recent_data(s, 'brazil').Close
            elif types[i] == 'etf':
                prices[s] = ip.get_etf_recent_data(s, 'brazil').Close
            else:
                pass
        else:
            if types[i] == 'stock':
                prices[s] = ip.get_stock_historical_data(s, 'brazil', from_date=from_date, to_date=to_date).Close
            elif types[i] == 'etf':
                prices[s] = ip.get_etf_historical_data(s, 'brazil', from_date=from_date, to_date=to_date).Close
            else:
                pass
    
    return(prices)

In [4]:
#== Felipe ==#
stocks = ['PETR4', 'MGLU3', 'BBAS3', 'CYRE3', 'Fundo de Invest Ishares SP 500']
amounts = [600, 400, 300, 200, 120]
types = ['stock','stock','stock','stock','etf']
df = get_prices(stocks, amounts, types, from_date='15/03/2020')

5it [00:03,  1.44it/s]


In [5]:
#== Benchmark == #
benchmark = df['Fundo de Invest Ishares SP 500'].pct_change()
benchmark

Date
2020-03-16         NaN
2020-03-17    0.056338
2020-03-18   -0.021978
2020-03-19   -0.016854
2020-03-20   -0.047619
                ...   
2020-12-28    0.020162
2020-12-29   -0.007092
2020-12-30    0.000190
2021-01-04    0.003618
2021-01-05    0.009962
Name: Fundo de Invest Ishares SP 500, Length: 201, dtype: float64

In [6]:
df['portfolio'] = df.apply(lambda x: x.dot(amounts)/sum(amounts), axis=1)

In [7]:
df.head()

,PETR4,MGLU3,BBAS3,CYRE3,Fundo de Invest Ishares SP 500,portfolio
Date,,,,,,
2020-03-16,13.09,8.50,29.63,16.06,129.22,23.988519
2020-03-17,13.00,8.87,31.07,15.86,136.50,24.827778
2020-03-18,11.29,7.19,25.94,12.59,133.50,22.203704
2020-03-19,12.21,7.62,24.93,13.84,131.25,22.451235
2020-03-20,12.00,7.65,23.98,13.34,125.00,21.680247


In [8]:
assert all(df.index == bench_returns.index)

NameError: name 'bench_returns' is not defined

In [ ]:
portfolio = df.portfolio.pct_change()

In [ ]:
%matplotlib inline
import quantstats as qs

In [ ]:
# show sharpe ratio
qs.stats.sharpe(portfolio)

In [ ]:
qs.plots.snapshot(portfolio, title='Portfolio Performance')

In [ ]:
qs.reports.full(portfolio, benchmark)